# 🎬 Federated API Governance — MCP + A2A Discovery Demo

## End-to-end demo: MCP servers, A2A agents, and federated API Center

This notebook demonstrates how Azure API Management + Azure API Center enable **federated governance** across distributed API environments — covering **MCP servers**, **A2A agents**, and **REST APIs** in a single unified catalog.

### Architecture
- **RG 2** — Pre-deployed REST→MCP conversion (Weather, Catalog, Order) + remote MCPs
- **RG 3** — FastMCP containers (Weather, Catalog, Order, Calculator) deployed from code  
- **RG 4** — A2A agents (Outline, Title, Summary) + REST APIs + 9 remote MCPs
- **RG 1** — **Central API Center** aggregating all APIs from RG 2, 3, 4

### Demo story
| Part | Topic | Description |
|------|-------|-------------|
| 1-3 | **MCP Servers** | Explore existing APIs, deploy FastMCP containers, test |
| 4-5 | **Federation** | Deploy central APIC, aggregate from RG 2 + RG 3 |
| 6-7 | **MCP Discovery** | Unified catalog, dynamic invoke, cross-env agent workflow |
| 8-9 | **A2A Agents** | Explore A2A agents, aggregate to central, agent card discovery |
| 10 | **Unified Governance** | Full catalog: REST + MCP + A2A, metrics summary |

### Prerequisites
- [Azure CLI](https://learn.microsoft.com/cli/azure/install-azure-cli) installed and signed in
- [Python 3.12+](https://www.python.org/) with the project requirements installed
- An Azure subscription with **Contributor** access

---
## Part 1 — Explore Existing Distributed APIs (RG 2)
---

> RG 2 is **already deployed** (from another lab) — it has 4 REST APIs converted to MCP servers + remote MCPs registered in API Center.

### 0️⃣ Initialize Variables

In [95]:
import os, sys, json, requests
sys.path.insert(1, '../../shared')
import utils

# Ensure correct subscription context
target_sub = "31613fe0-1e9b-4a97-b771-dc48fbaa0fbb"
utils.run(f'az account set --subscription {target_sub}', "", "")

# ═══════════════════════════════════════════════════════════
#  🔢 DEPLOYMENT INDEX — change this to spin up a new demo
#     while the previous one is still deleting
# ═══════════════════════════════════════════════════════════
IDX = 2   # increment for each new demo run (1, 2, 3, …)

# ── RG 2: Existing mixed REST→MCP deployment (read-only, from another lab) ──
rg2_name = "rg-lab-mcp-demo-1-1"
rg2_apic = "apic-demo-1-xbilmxrmx74wu"
rg2_apim = "apim-mcp-demo-1"

# ── RG 3: New FastMCP containers deployment ──
rg3_name = f"rg-lab-mcp-containers-{IDX}"
rg3_location = "uksouth"
rg3_apim_name = f"apim-fastmcp-{IDX}"
rg3_apic_prefix = f"apic-fastmcp-{IDX}"
rg3_deployment_name = "fastmcp-containers"

# ── RG 4: A2A agents deployment (read-only, from another lab) ──
rg4_name = "rg-lab-a2a-demo"
rg4_apic = "apic-a2a-v2vnj5h4ckvgy"
rg4_apim = "apim-a2a-wihxr7"
rg4_apim_gateway = "https://apim-a2a-wihxr7.azure-api.net"

# ── RG 1: Central API Center ──
rg1_name = f"rg-lab-apic-central-{IDX}"
rg1_location = "swedencentral"
rg1_apic_name = f"apic-central-{IDX}"
rg1_deployment_name = "central-apic"

utils.print_ok(f"Variables initialized  (deployment index = {IDX})")
print(f"  RG 1 (Central APIC):   {rg1_name} ({rg1_location})")
print(f"  RG 2 (Mixed APIs):     {rg2_name}")
print(f"  RG 3 (FastMCP):        {rg3_name} ({rg3_location})")
print(f"  RG 4 (A2A Agents):     {rg4_name}")

⚙️ Running: az account set --subscription 31613fe0-1e9b-4a97-b771-dc48fbaa0fbb 
✅ Variables initialized  (deployment index = 2) ⌚ 12:09:17.590149 
  RG 1 (Central APIC):   rg-lab-apic-central-2 (swedencentral)
  RG 2 (Mixed APIs):     rg-lab-mcp-demo-1-1
  RG 3 (FastMCP):        rg-lab-mcp-containers-2 (uksouth)
  RG 4 (A2A Agents):     rg-lab-a2a-demo


### 1️⃣ List APIs in RG 2 — existing distributed API Center

This API Center was deployed previously. It has a mix of:
- **REST APIs** converted to **MCP servers** via APIM policies
- **Remote 3rd-party MCP servers** (Atlassian, Sentry, PayPal, etc.) registered for discovery

In [21]:
# List all APIs in RG 2's API Center
output = utils.run(
    f'az apic api list -g {rg2_name} -n {rg2_apic} --query "[].{{Name:name, Title:title, Kind:kind}}" -o table',
    "Listed APIs in RG 2 API Center", "Failed to list APIs")
if output.success:
    print(output.text)

# Snapshot for later comparison
output = utils.run(f'az apic api list -g {rg2_name} -n {rg2_apic} -o json', "", "")
if output.success and output.json_data:
    rg2_apis = output.json_data
    rg2_rest = sum(1 for api in rg2_apis if api.get('kind') == 'rest')
    rg2_mcp  = sum(1 for api in rg2_apis if api.get('kind') == 'mcp')
    print(f"\n📊 RG 2 Total: {len(rg2_apis)} APIs — {rg2_rest} REST, {rg2_mcp} MCP")

⚙️ Running: az apic api list -g rg-lab-mcp-demo-1-1 -n apic-demo-1-xbilmxrmx74wu --query "[].{Name:name, Title:title, Kind:kind}" -o table 
✅ Listed APIs in RG 2 API Center ⌚ 21:33:10.331402 :9s]
Name              Title                Kind
----------------  -------------------  ------
sentry            Sentry               mcp
cloudflare        Cloudflare           mcp
atlassian         Atlassian            mcp
paypal            Paypal               mcp
linear            Linear               mcp
intercom          Intercom             mcp
square            Square               mcp
asana             Asana                mcp
plaid             Plaid                mcp
swagger-petstore  Swagger Petstore     rest
weather-api       Weather API          rest
order-api         Place Order API      rest
catalog-api       Product Catalog API  rest
order-mcp         Place Order MCP      mcp
weather-mcp       Weather MCP          mcp
catalog-mcp       Product Catalog MCP  mcp
calculator-api    Calc

### 2️⃣ Show APIM APIs in RG 2 — including MCP type APIs

In [22]:
# Use ARM REST API to see MCP-type APIs (az apim api list doesn't show 'type' field)
subscription_id = utils.get_current_subscription()

output = utils.run(
    f'az rest --method GET '
    f'--url "https://management.azure.com/subscriptions/{subscription_id}/resourceGroups/{rg2_name}'
    f'/providers/Microsoft.ApiManagement/service/{rg2_apim}/apis?api-version=2024-06-01-preview" '
    f'--query "value[].{{Name:name, DisplayName:properties.displayName, Type:properties.type, Path:properties.path}}" -o table',
    f"Listed APIs on APIM '{rg2_apim}'", "Failed")
if output.success:
    print(output.text)

⚙️ Running: az account show 
✅ Retrieved az account ⌚ 21:33:26.464608 :4s]
👉🏽 Using Subscription ID: 31613fe0-1e9b-4a97-b771-dc48fbaa0fbb (ME-MngEnvMCAP219373-ncheruvu-1)
⚙️ Running: az rest --method GET --url "https://management.azure.com/subscriptions/31613fe0-1e9b-4a97-b771-dc48fbaa0fbb/resourceGroups/rg-lab-mcp-demo-1-1/providers/Microsoft.ApiManagement/service/apim-mcp-demo-1/apis?api-version=2024-06-01-preview" --query "value[].{Name:name, DisplayName:properties.displayName, Type:properties.type, Path:properties.path}" -o table 
✅ Listed APIs on APIM 'apim-mcp-demo-1' ⌚ 21:33:31.560783 :5s]
Name            DisplayName          Path            Type
--------------  -------------------  --------------  ------
calculator-api  Calculator API       calculator
calculator-mcp  Calculator MCP       calculator-mcp  mcp
order-api       Place Order API      order
order-mcp       Place Order MCP      order-mcp       mcp
catalog-api     Product Catalog API  catalog
catalog-mcp     Product Cata

---
## Part 2 — Deploy FastMCP Containers (RG 3)
---

> Deploy 4 containerized FastMCP servers into a **separate resource group** — simulating a different team/subscription.
> Each server is: Container App → APIM MCP Proxy → API Center registration.

### 3️⃣ Deploy infrastructure + containers via Bicep

| Layer | Resources |
|-------|-----------|
| **Monitoring** | Log Analytics, Application Insights, MCP Dashboard |
| **API Gateway** | API Management (Basicv2) with MCP diagnostics |
| **API Governance** | API Center with MCP environment |
| **Containers** | ACR + ACA Environment + 4 Container Apps |
| **MCP Proxies** | 4× Streamable MCP APIs in APIM |
| **Discoverability** | 4× API Center registrations with VS Code install links |

> ⏱️ First deployment takes ~5-8 minutes (APIM provisioning). Subsequent runs are incremental.

In [84]:
# Create RG 3 and deploy infrastructure
utils.create_resource_group(rg3_name, rg3_location)

apim_subscriptions_config = [{"name": "subscription1", "displayName": "Subscription 1"}]

bicep_parameters = {
    "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
    "contentVersion": "1.0.0.0",
    "parameters": {
        "apimSku": { "value": "Basicv2" },
        "apimName": { "value": rg3_apim_name },
        "apimSubscriptionsConfig": { "value": apim_subscriptions_config },
        "apicLocation": { "value": rg3_location },
        "apicServiceNamePrefix": { "value": rg3_apic_prefix }
    }
}

with open('params-demo-containers.json', 'w') as f:
    f.write(json.dumps(bicep_parameters))

output = utils.run(
    f"az deployment group create --name {rg3_deployment_name} --resource-group {rg3_name} "
    f"--template-file demo-mcp-containers.bicep --parameters params-demo-containers.json",
    f"✅ RG 3 deployment '{rg3_deployment_name}' succeeded",
    f"❌ RG 3 deployment failed"
)

⚙️ Running: az group show --name rg-lab-mcp-containers-2 
👉🏽 Resource group rg-lab-mcp-containers-2 does not yet exist. Creating the resource group now...
⚙️ Running: az group create --name rg-lab-mcp-containers-2 --location uksouth --tags source=ai-gateway 
✅ Resource group 'rg-lab-mcp-containers-2' created ⌚ 11:50:47.072433 :13s]
⚙️ Running: az deployment group create --name fastmcp-containers --resource-group rg-lab-mcp-containers-2 --template-file demo-mcp-containers.bicep --parameters params-demo-containers.json 
❌ ❌ RG 3 deployment failed ⌚ 11:53:17.843961 :30s] WARNING: A new Bicep release is available: v0.40.2. Upgrade now by running "az bicep upgrade".

c:\Git\AZ\apic-discovery\modules\apim-streamable-mcp\api.bicep(30,5) : Warning BCP037: The property "backendId" is not allowed on objects of type "ApiCreateOrUpdatePropertiesOrApiContractProperties". Permissible properties include "apiRevision", "apiRevisionDescription", "apiType", "apiVersion", "apiVersionDescription", "apiVer

### 4️⃣ Retrieve deployment outputs

In [87]:
# Try deployment outputs first; fall back to direct resource queries
output = utils.run(f"az deployment group show --name {rg3_deployment_name} -g {rg3_name}", "", "")

has_outputs = False
if output.success and output.json_data:
    outputs = output.json_data.get('properties', {}).get('outputs')
    if outputs:
        has_outputs = True

if has_outputs:
    rg3_apim_service  = utils.get_deployment_output(output, 'apimServiceName', 'APIM Service')
    rg3_gateway_url   = utils.get_deployment_output(output, 'apimGatewayUrl', 'APIM Gateway')
    rg3_apic          = utils.get_deployment_output(output, 'apicServiceName', 'API Center')
    rg3_acr_name      = utils.get_deployment_output(output, 'acrName', 'ACR Name')
    rg3_acr_server    = utils.get_deployment_output(output, 'acrLoginServer', 'ACR Server')
    rg3_weather_url   = utils.get_deployment_output(output, 'weatherMcpUrl', 'Weather Container')
    rg3_catalog_url   = utils.get_deployment_output(output, 'catalogMcpUrl', 'Catalog Container')
    rg3_order_url     = utils.get_deployment_output(output, 'orderMcpUrl', 'Order Container')
    rg3_calculator_url = utils.get_deployment_output(output, 'calculatorMcpUrl', 'Calculator Container')
    rg3_app_insights  = utils.get_deployment_output(output, 'appInsightsName', 'App Insights')
    rg3_subscriptions = json.loads(utils.get_deployment_output(output, 'apimSubscriptions').replace("\'", "\""))
    rg3_api_key = rg3_subscriptions[0].get("key")
else:
    utils.print_info("Deployment outputs unavailable — resolving from live resources")

    rg3_apim_service = rg3_apim_name
    o = utils.run(f'az apim show --name {rg3_apim_name} -g {rg3_name} --query gatewayUrl -o tsv', "APIM Gateway", "")
    rg3_gateway_url = o.text.strip() if o.success else ""

    o = utils.run(f'az apic list -g {rg3_name} --query "[0].name" -o tsv', "API Center", "")
    rg3_apic = o.text.strip() if o.success else ""

    o = utils.run(f'az acr list -g {rg3_name} --query "[0].{{n:name,s:loginServer}}" -o json', "ACR", "")
    if o.success and o.json_data:
        rg3_acr_name = o.json_data["n"]
        rg3_acr_server = o.json_data["s"]

    # Container URLs — may be empty if apps haven't started yet
    for app_name, var_name in [("weather-mcp", "rg3_weather_url"), ("catalog-mcp", "rg3_catalog_url"),
                                ("order-mcp", "rg3_order_url"), ("calculator-mcp", "rg3_calculator_url")]:
        o = utils.run(f'az containerapp show --name {app_name} -g {rg3_name} --query "properties.configuration.ingress.fqdn" -o tsv', "", "")
        fqdn = o.text.strip() if o.success and o.text.strip() else ""
        globals()[var_name] = f"https://{fqdn}" if fqdn else ""

    o = utils.run(f'az monitor app-insights component show -g {rg3_name} --query "[0].name" -o tsv', "App Insights", "")
    rg3_app_insights = o.text.strip() if o.success else ""

    sub_id = utils.get_current_subscription()
    o = utils.run(
        f'az rest --method POST --url "https://management.azure.com/subscriptions/{sub_id}'
        f'/resourceGroups/{rg3_name}/providers/Microsoft.ApiManagement/service/{rg3_apim_name}'
        f'/subscriptions/subscription1/listSecrets?api-version=2022-08-01" --query primaryKey -o tsv',
        "API Key", "")
    rg3_api_key = o.text.strip() if o.success else ""

print(f"\n📋 RG 3 Resources:")
print(f"  APIM:     {rg3_apim_service}")
print(f"  Gateway:  {rg3_gateway_url}")
print(f"  APIC:     {rg3_apic}")
print(f"  ACR:      {rg3_acr_name} ({rg3_acr_server})")
print(f"  API Key:  ****{rg3_api_key[-4:] if rg3_api_key else '????'}")

⚙️ Running: az deployment group show --name fastmcp-containers -g rg-lab-mcp-containers-2 
👉🏽 APIM Service: apim-fastmcp-2
👉🏽 APIM Gateway: https://apim-fastmcp-2.azure-api.net
👉🏽 API Center: apic-fastmcp-2-brasrj7hswmuc
👉🏽 ACR Name: acrbrasrj7hswmuc
👉🏽 ACR Server: acrbrasrj7hswmuc.azurecr.io
👉🏽 Weather Container: https://weather-mcp.agreeablecliff-f1e0b4f8.uksouth.azurecontainerapps.io
👉🏽 Catalog Container: https://catalog-mcp.agreeablecliff-f1e0b4f8.uksouth.azurecontainerapps.io
👉🏽 Order Container: https://order-mcp.agreeablecliff-f1e0b4f8.uksouth.azurecontainerapps.io
👉🏽 Calculator Container: https://calculator-mcp.agreeablecliff-f1e0b4f8.uksouth.azurecontainerapps.io
👉🏽 App Insights: insights-brasrj7hswmuc

📋 RG 3 Resources:
  APIM:     apim-fastmcp-2
  Gateway:  https://apim-fastmcp-2.azure-api.net
  APIC:     apic-fastmcp-2-brasrj7hswmuc
  ACR:      acrbrasrj7hswmuc (acrbrasrj7hswmuc.azurecr.io)
  API Key:  ****5f38


### 5️⃣ Build and push container images to ACR

In [63]:
containers = [
    ("weather-mcp",    "src/weather/container"),
    ("catalog-mcp",    "src/product-catalog/container"),
    ("order-mcp",      "src/place-order/container"),
    ("calculator-mcp", "src/calculator/container"),
]

for image_name, context_dir in containers:
    output = utils.run(
        f"az acr build --registry {rg3_acr_name} --image {image_name}:latest "
        f"--file {context_dir}/Dockerfile {context_dir} --no-logs",
        f"✅ Built {image_name}", f"❌ Failed to build {image_name}")
    if not output.success:
        break

⚙️ Running: az acr build --registry acrbrasrj7hswmuc --image weather-mcp:latest --file src/weather/container/Dockerfile src/weather/container --no-logs 
✅ ✅ Built weather-mcp ⌚ 11:13:08.539716 :15s]
⚙️ Running: az acr build --registry acrbrasrj7hswmuc --image catalog-mcp:latest --file src/product-catalog/container/Dockerfile src/product-catalog/container --no-logs 
✅ ✅ Built catalog-mcp ⌚ 11:14:24.978340 :16s]
⚙️ Running: az acr build --registry acrbrasrj7hswmuc --image order-mcp:latest --file src/place-order/container/Dockerfile src/place-order/container --no-logs 
✅ ✅ Built order-mcp ⌚ 11:15:38.976795 :13s]
⚙️ Running: az acr build --registry acrbrasrj7hswmuc --image calculator-mcp:latest --file src/calculator/container/Dockerfile src/calculator/container --no-logs 
✅ ✅ Built calculator-mcp ⌚ 11:16:38.216214 :59s]


### 6️⃣ Update container apps with built images

In [64]:
for image_name, _ in containers:
    output = utils.run(
        f"az containerapp update --name {image_name} --resource-group {rg3_name} "
        f"--image {rg3_acr_server}/{image_name}:latest",
        f"✅ Updated {image_name}", f"❌ Failed to update {image_name}")
    if not output.success:
        break

⚙️ Running: az containerapp update --name weather-mcp --resource-group rg-lab-mcp-containers-2 --image acrbrasrj7hswmuc.azurecr.io/weather-mcp:latest 
✅ ✅ Updated weather-mcp ⌚ 11:18:03.639180 :2s]
⚙️ Running: az containerapp update --name catalog-mcp --resource-group rg-lab-mcp-containers-2 --image acrbrasrj7hswmuc.azurecr.io/catalog-mcp:latest 
✅ ✅ Updated catalog-mcp ⌚ 11:19:12.618274 :8s]
⚙️ Running: az containerapp update --name order-mcp --resource-group rg-lab-mcp-containers-2 --image acrbrasrj7hswmuc.azurecr.io/order-mcp:latest 
✅ ✅ Updated order-mcp ⌚ 11:20:01.021547 :48s]
⚙️ Running: az containerapp update --name calculator-mcp --resource-group rg-lab-mcp-containers-2 --image acrbrasrj7hswmuc.azurecr.io/calculator-mcp:latest 
✅ ✅ Updated calculator-mcp ⌚ 11:20:30.839088 :29s]


---
## Part 3 — Test FastMCP Containers
---

### 🧪 Test all 4 containers — direct and through APIM

MCP Streamable HTTP protocol: POST JSON-RPC → SSE response stream

In [88]:
def mcp_initialize(endpoint, headers=None):
    """Initialize an MCP session and return the session ID."""
    request = {
        "method": "initialize",
        "params": {
            "protocolVersion": "2025-03-26",
            "capabilities": {},
            "clientInfo": {"name": "demo-client", "version": "1.0"}
        },
        "jsonrpc": "2.0", "id": 0
    }
    h = {"Content-Type": "application/json", "Accept": "application/json, text/event-stream"}
    if headers:
        h.update(headers)
    try:
        resp = requests.post(endpoint, json=request, headers=h, timeout=30)
        session_id = resp.headers.get("Mcp-Session-Id", "")
        resp.close()
        return session_id
    except Exception:
        return ""

def call_mcp(endpoint, tool_name, arguments, label="", extra_headers=None):
    """Call an MCP tool (with auto-initialize) and return the parsed result."""
    h = {"Content-Type": "application/json", "Accept": "application/json, text/event-stream"}
    if extra_headers:
        h.update(extra_headers)

    # Initialize to get session
    session_id = mcp_initialize(endpoint, extra_headers)
    if session_id:
        h["Mcp-Session-Id"] = session_id

    request = {
        "method": "tools/call",
        "params": {"name": tool_name, "arguments": arguments},
        "jsonrpc": "2.0", "id": 1
    }
    response = None
    try:
        response = requests.post(endpoint, json=request, stream=True, timeout=30, headers=h)
        if response.status_code == 200:
            for line in response.iter_lines(decode_unicode=True):
                if line and line.startswith("data:"):
                    data = json.loads(line[5:].strip())
                    if "result" in data and "content" in data["result"]:
                        text = data["result"]["content"][0].get("text", "")
                        try:
                            result = json.loads(text)
                        except json.JSONDecodeError:
                            result = text
                        utils.print_ok(f"  {label}: ✅")
                        return result
            utils.print_ok(f"  {label}: ✅ (no data in stream)")
            return None
        else:
            utils.print_error(f"  {label}: HTTP {response.status_code}")
            return None
    except Exception as e:
        utils.print_error(f"  {label}: {e}")
        return None
    finally:
        if response:
            response.close()

def list_mcp_tools(endpoint, label="", extra_headers=None):
    """List available tools on an MCP endpoint."""
    h = {"Content-Type": "application/json", "Accept": "application/json, text/event-stream"}
    if extra_headers:
        h.update(extra_headers)

    session_id = mcp_initialize(endpoint, extra_headers)
    if session_id:
        h["Mcp-Session-Id"] = session_id

    request = {"method": "tools/list", "params": {}, "jsonrpc": "2.0", "id": 1}
    response = None
    try:
        response = requests.post(endpoint, json=request, stream=True, timeout=30, headers=h)
        if response.status_code == 200:
            for line in response.iter_lines(decode_unicode=True):
                if line and line.startswith("data:"):
                    data = json.loads(line[5:].strip())
                    if "result" in data and "tools" in data["result"]:
                        tools = [t["name"] for t in data["result"]["tools"]]
                        print(f"  {label}: {', '.join(tools)}")
                        return tools
        return []
    except Exception as e:
        utils.print_error(f"  {label}: {e}")
        return []
    finally:
        if response:
            response.close()

utils.print_ok("MCP helper functions defined (with session support)")

✅ MCP helper functions defined (with session support) ⌚ 11:58:37.196336 


In [89]:
# Test each container — direct call
print("🧪 Direct container tests")
print("=" * 50)

tests = [
    (f"{rg3_weather_url}/weather/mcp",      "get_weather",       {"city": "London"},             "Weather"),
    (f"{rg3_catalog_url}/catalog/mcp",       "search_products",   {"query": "laptop"},            "Catalog"),
    (f"{rg3_order_url}/order/mcp",           "place_order",       {"product_id": "PROD-001", "quantity": 1}, "Order"),
    (f"{rg3_calculator_url}/calculator/mcp", "calculate",         {"operation": "multiply", "a": 7, "b": 8}, "Calculator"),
]

results = {}
for endpoint, tool, args, label in tests:
    result = call_mcp(endpoint, tool, args, f"{label} (direct)")
    results[f"{label}_direct"] = result is not None
    if result:
        print(f"    → {json.dumps(result, indent=2)[:200]}")

# Test through APIM (backend URL already includes /mcp, so no extra /mcp needed)
print(f"\n🧪 APIM proxy tests ({rg3_gateway_url})")
print("=" * 50)
apim_headers = {"api-key": rg3_api_key}

apim_tests = [
    (f"{rg3_gateway_url}/weather-mcp",    "get_weather",       {"city": "Paris"},              "Weather"),
    (f"{rg3_gateway_url}/catalog-mcp",     "search_products",   {"query": "phone"},             "Catalog"),
    (f"{rg3_gateway_url}/order-mcp",       "place_order",       {"product_id": "PROD-002", "quantity": 3}, "Order"),
    (f"{rg3_gateway_url}/calculator-mcp",  "calculate",         {"operation": "add", "a": 42, "b": 17}, "Calculator"),
]

for endpoint, tool, args, label in apim_tests:
    result = call_mcp(endpoint, tool, args, f"{label} (APIM)", extra_headers=apim_headers)
    results[f"{label}_apim"] = result is not None
    if result:
        print(f"    → {json.dumps(result, indent=2)[:200]}")

passed = sum(1 for v in results.values() if v)
print(f"\n📊 Results: {passed}/{len(results)} tests passed")

🧪 Direct container tests
✅   Weather (direct): ✅ ⌚ 11:58:42.398764 
    → "{'city': 'London', 'condition': 'Windy', 'temperature': 9.66, 'humidity': 81.01}"
✅   Catalog (direct): ✅ ⌚ 11:58:44.536999 
    → "[{'id': 'PROD-006', 'name': 'Laptop Backpack', 'category': 'Accessories', 'price': 59.99, 'stock': 300, 'description': 'Water-resistant laptop backpack with USB charging port'}]"
✅   Order (direct): ✅ ⌚ 11:58:46.475637 
    → "{'order_id': 'ORD-B079EB77', 'product_id': 'PROD-001', 'product_name': 'Wireless Mouse', 'quantity': 1, 'unit_price': 29.99, 'total': 29.99, 'status': 'confirmed', 'created_at': '2026-02-10T16:58:46.
✅   Calculator (direct): ✅ ⌚ 11:58:48.256854 
    → "{'operation': 'multiply', 'a': 7.0, 'b': 8.0, 'result': 56.0}"

🧪 APIM proxy tests (https://apim-fastmcp-2.azure-api.net)
❌   Weather (APIM): HTTPSConnectionPool(host='apim-fastmcp-2.azure-api.net', port=443): Read timed out. (read timeout=30) ⌚ 11:59:19.768312 
✅   Catalog (APIM): ✅ ⌚ 11:59:22.777284 
    → "[{

### 7️⃣ Verify RG 3's local API Center

Each container MCP server was auto-registered in RG 3's own API Center during Bicep deployment.

In [69]:
output = utils.run(
    f'az apic api list -g {rg3_name} -n {rg3_apic} --query "[].{{Name:name, Title:title, Kind:kind}}" -o table',
    "Listed APIs in RG 3 API Center", "Failed")
if output.success:
    print(output.text)

# Snapshot
output = utils.run(f'az apic api list -g {rg3_name} -n {rg3_apic} -o json', "", "")
if output.success and output.json_data:
    rg3_apis = output.json_data
    rg3_mcp = sum(1 for api in rg3_apis if api.get('kind') == 'mcp')
    print(f"\n📊 RG 3 Total: {len(rg3_apis)} APIs — {rg3_mcp} MCP Servers")

⚙️ Running: az apic api list -g rg-lab-mcp-containers-2 -n apic-fastmcp-2-brasrj7hswmuc --query "[].{Name:name, Title:title, Kind:kind}" -o table 
✅ Listed APIs in RG 3 API Center ⌚ 11:23:27.523978 :3s]
Name              Title                Kind
----------------  -------------------  ------
sentry            Sentry               mcp
intercom          Intercom             mcp
asana             Asana                mcp
plaid             Plaid                mcp
atlassian         Atlassian            mcp
linear            Linear               mcp
cloudflare        Cloudflare           mcp
paypal            Paypal               mcp
square            Square               mcp
swagger-petstore  Swagger Petstore     rest
order-mcp         Order Service MCP    mcp
calculator-mcp    Calculator MCP       mcp
catalog-mcp       Product Catalog MCP  mcp
weather-mcp       Weather MCP          mcp

⚙️ Running: az apic api list -g rg-lab-mcp-containers-2 -n apic-fastmcp-2-brasrj7hswmuc -o json 

📊 RG 

---
## Part 4 — Deploy Central API Center (RG 1)
---

> 💡 **This is the key architecture moment.**  
> The central API Center acts as the **single pane of glass** for all APIs and MCP servers across the organization — regardless of which resource group, subscription, or APIM instance hosts them.

In [70]:
# Create RG 1 and deploy central API Center
utils.create_resource_group(rg1_name, rg1_location)

bicep_parameters = {
    "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
    "contentVersion": "1.0.0.0",
    "parameters": {
        "apicServiceName": { "value": rg1_apic_name },
        "location": { "value": rg1_location }
    }
}

with open('params-central-apic.json', 'w') as f:
    f.write(json.dumps(bicep_parameters))

output = utils.run(
    f"az deployment group create --name {rg1_deployment_name} --resource-group {rg1_name} "
    f"--template-file demo-central-apic.bicep --parameters params-central-apic.json",
    f"✅ Central API Center deployed in {rg1_location}",
    f"❌ Central API Center deployment failed"
)

if output.success and output.json_data:
    rg1_apic_actual = utils.get_deployment_output(output, 'name', 'Central APIC Name')
    utils.print_info(f"Central APIC: {rg1_apic_actual}")

⚙️ Running: az group show --name rg-lab-apic-central-2 
👉🏽 Resource group rg-lab-apic-central-2 does not yet exist. Creating the resource group now...
⚙️ Running: az group create --name rg-lab-apic-central-2 --location swedencentral --tags source=ai-gateway 
✅ Resource group 'rg-lab-apic-central-2' created ⌚ 11:23:49.694250 :8s]
⚙️ Running: az deployment group create --name central-apic --resource-group rg-lab-apic-central-2 --template-file demo-central-apic.bicep --parameters params-central-apic.json 
✅ ✅ Central API Center deployed in swedencentral ⌚ 11:24:34.323079 :44s]


---
## Part 5 — Aggregate APIs into Central Catalog
---

> Pull APIs from **both distributed API Centers** into the central catalog.  
> This simulates a platform team aggregating APIs from multiple product teams.

In [90]:
import subprocess

def get_apic_apis(resource_group, apic_name):
    """Fetch all APIs from an API Center instance."""
    output = utils.run(f'az apic api list -g {resource_group} -n {apic_name} -o json', "", "")
    if output.success and output.json_data:
        return output.json_data
    return []

def get_apic_deployments(resource_group, apic_name, api_name):
    """Fetch deployments for an API to get the runtimeUri."""
    output = utils.run(
        f'az apic api deployment list -g {resource_group} -n {apic_name} --api-id {api_name} -o json', "", "")
    if output.success and output.json_data:
        return output.json_data
    return []

def create_api_via_arm(api_name, kind, title, description):
    """Create an API in central APIC using ARM REST API (supports mcp/a2a kinds)."""
    url = (f"https://management.azure.com/subscriptions/{subscription_id}/resourceGroups/{rg1_name}"
           f"/providers/Microsoft.ApiCenter/services/{rg1_apic_name}/workspaces/default/apis/{api_name}"
           f"?api-version=2024-06-01-preview")
    body = json.dumps({"properties": {"title": title, "kind": kind, "description": description[:200]}})
    body_file = f"_temp_api_{api_name}.json"
    with open(body_file, 'w') as f:
        f.write(body)
    output = utils.run(f'az rest --method PUT --url "{url}" --body @{body_file}', "", "")
    try:
        os.remove(body_file)
    except:
        pass
    return output.success

def create_deployment_via_subprocess(api_name, deployment_id, title, env_name, runtime_uris):
    """Create an API deployment using subprocess to handle complex JSON escaping."""
    server_json = json.dumps({"runtimeUri": runtime_uris})
    args = [
        "az", "apic", "api", "deployment", "create",
        "-g", rg1_name, "-n", rg1_apic_name,
        "--api-id", api_name,
        "--deployment-id", deployment_id,
        "--title", title,
        "--environment-id", f"/workspaces/default/environments/{env_name}",
        "--definition-id", f"/workspaces/default/apis/{api_name}/versions/1-0-0/definitions/default",
        "--server", server_json
    ]
    result = subprocess.run(args, capture_output=True, text=True, shell=True)
    return result.returncode == 0

def register_api_in_central(api, source_rg, source_apic, source_label):
    """Register a single API from a distributed APIC into the central APIC."""
    api_name = api['name']
    kind = api.get('kind', 'rest')
    title = api.get('title', api_name)
    description = api.get('description', '') or api.get('summary', '') or f'{title} from {source_label}'
    env_map = {'mcp': 'mcp', 'a2a': 'a2a', 'rest': 'api'}
    env_name = env_map.get(kind, 'api')

    # CLI --type only supports rest/graphql/grpc/soap. Use ARM REST API for mcp/a2a.
    cli_type_map = {'rest': 'rest', 'graphql': 'graphql', 'grpc': 'grpc', 'soap': 'soap'}
    if kind in cli_type_map:
        cmd = (f'az apic api create -g {rg1_name} -n {rg1_apic_name} --api-id {api_name} '
               f'--title "{title}" --type {cli_type_map[kind]} '
               f'--description "{description[:200]}" ')
        utils.run(cmd, "", "")
    else:
        create_api_via_arm(api_name, kind, title, description)

    # Add version
    utils.run(f'az apic api version create -g {rg1_name} -n {rg1_apic_name} --api-id {api_name} '
              f'--version-id 1-0-0 --title "1.0.0" --lifecycle-stage production', "", "")

    # Add definition
    utils.run(f'az apic api definition create -g {rg1_name} -n {rg1_apic_name} --api-id {api_name} '
              f'--version-id 1-0-0 --definition-id default --title "Default"', "", "")

    # Get runtimeUri from source deployments
    deployments = get_apic_deployments(source_rg, source_apic, api_name)
    runtime_uris = []
    for dep in deployments:
        uris = dep.get('server', {}).get('runtimeUri', [])
        runtime_uris.extend(uris)

    # Register deployment with subprocess to handle JSON escaping
    if runtime_uris:
        deploy_id = f"{api_name}-from-{source_label}"
        deploy_title = f"{title} ({source_label})"
        create_deployment_via_subprocess(api_name, deploy_id, deploy_title, env_name, runtime_uris)

    return runtime_uris

utils.print_ok("Aggregation functions defined (supports REST, MCP, and A2A via ARM REST API)")

✅ Aggregation functions defined (supports REST, MCP, and A2A via ARM REST API) ⌚ 11:59:38.343779 


### 8️⃣ Sync APIs from RG 2 (mixed REST→MCP) → Central APIC

In [72]:
# Aggregate from RG 2 (mixed REST→MCP APIs)
rg2_apis = get_apic_apis(rg2_name, rg2_apic)
print(f"📥 Syncing {len(rg2_apis)} APIs from RG 2 ({rg2_apic}) → Central APIC")
print("=" * 60)

rg2_synced = 0
for api in rg2_apis:
    uris = register_api_in_central(api, rg2_name, rg2_apic, "rg2")
    uri_str = uris[0] if uris else "—"
    print(f"  ✅ {api['name']:25} ({api.get('kind','?'):4}) → {uri_str}")
    rg2_synced += 1

print(f"\n📊 Synced {rg2_synced} APIs from RG 2")

⚙️ Running: az apic api list -g rg-lab-mcp-demo-1-1 -n apic-demo-1-xbilmxrmx74wu -o json 
📥 Syncing 0 APIs from RG 2 (apic-demo-1-xbilmxrmx74wu) → Central APIC

📊 Synced 0 APIs from RG 2


### 9️⃣ Sync APIs from RG 3 (FastMCP containers) → Central APIC

In [73]:
# Aggregate from RG 3 (FastMCP containers)
rg3_apis = get_apic_apis(rg3_name, rg3_apic)
print(f"📥 Syncing {len(rg3_apis)} APIs from RG 3 ({rg3_apic}) → Central APIC")
print("=" * 60)

rg3_synced = 0
for api in rg3_apis:
    uris = register_api_in_central(api, rg3_name, rg3_apic, "rg3")
    uri_str = uris[0] if uris else "—"
    print(f"  ✅ {api['name']:25} ({api.get('kind','?'):4}) → {uri_str}")
    rg3_synced += 1

print(f"\n📊 Synced {rg3_synced} APIs from RG 3")

⚙️ Running: az apic api list -g rg-lab-mcp-containers-2 -n apic-fastmcp-2-brasrj7hswmuc -o json 
📥 Syncing 14 APIs from RG 3 (apic-fastmcp-2-brasrj7hswmuc) → Central APIC
⚙️ Running: az rest --method PUT --url "https://management.azure.com/subscriptions/31613fe0-1e9b-4a97-b771-dc48fbaa0fbb/resourceGroups/rg-lab-apic-central-2/providers/Microsoft.ApiCenter/services/apic-central-2/workspaces/default/apis/sentry?api-version=2024-06-01-preview" --body @_temp_api_sentry.json 
⚙️ Running: az apic api version create -g rg-lab-apic-central-2 -n apic-central-2 --api-id sentry --version-id 1-0-0 --title "1.0.0" --lifecycle-stage production 
⚙️ Running: az apic api definition create -g rg-lab-apic-central-2 -n apic-central-2 --api-id sentry --version-id 1-0-0 --definition-id default --title "Default" 
⚙️ Running: az apic api deployment list -g rg-lab-mcp-containers-2 -n apic-fastmcp-2-brasrj7hswmuc --api-id sentry -o json 
  ✅ sentry                    (mcp ) → https://mcp.sentry.dev/sse
⚙️ Runni

---
## Part 6 — Unified Discovery from Central API Center
---

> 💡 **Single pane of glass** — query one API Center, discover APIs from all environments.

In [74]:
# List everything in the central API Center
output = utils.run(
    f'az apic api list -g {rg1_name} -n {rg1_apic_name} --query "[].{{Name:name, Title:title, Kind:kind}}" -o table',
    "Listed ALL APIs in Central API Center", "Failed")
if output.success:
    print(output.text)

output = utils.run(f'az apic api list -g {rg1_name} -n {rg1_apic_name} -o json', "", "")
if output.success and output.json_data:
    central_apis = output.json_data
    central_rest = sum(1 for api in central_apis if api.get('kind') == 'rest')
    central_mcp  = sum(1 for api in central_apis if api.get('kind') == 'mcp')
    central_a2a  = sum(1 for api in central_apis if api.get('kind') == 'a2a')
    _rg4 = rg4_synced if 'rg4_synced' in dir() else 0
    sources = f"RG 2 ({rg2_synced}) + RG 3 ({rg3_synced})"
    if _rg4 > 0:
        sources += f" + RG 4 ({_rg4})"
    print(f"\n📊 Central Catalog: {len(central_apis)} APIs — {central_rest} REST, {central_mcp} MCP, {central_a2a} A2A")
    print(f"   Aggregated from: {sources}")

⚙️ Running: az apic api list -g rg-lab-apic-central-2 -n apic-central-2 --query "[].{Name:name, Title:title, Kind:kind}" -o table 
✅ Listed ALL APIs in Central API Center ⌚ 11:31:38.080588 :7s]
Name              Title                Kind
----------------  -------------------  ------
swagger-petstore  Swagger Petstore     rest
sentry            Sentry               mcp
intercom          Intercom             mcp
asana             Asana                mcp
plaid             Plaid                mcp
atlassian         Atlassian            mcp
linear            Linear               mcp
cloudflare        Cloudflare           mcp
paypal            Paypal               mcp
square            Square               mcp
order-mcp         Order Service MCP    mcp
calculator-mcp    Calculator MCP       mcp
catalog-mcp       Product Catalog MCP  mcp
weather-mcp       Weather MCP          mcp

⚙️ Running: az apic api list -g rg-lab-apic-central-2 -n apic-central-2 -o json 

📊 Central Catalog: 14 APIs — 1

### 🔍 Filter: MCP servers with deployment endpoints from Central Catalog

In [75]:
# Show MCP servers from central catalog with their deployment endpoints
output = utils.run(
    f'az apic api list -g {rg1_name} -n {rg1_apic_name} --query "[?kind==\'mcp\'].{{Name:name, Title:title, Kind:kind}}" -o table',
    "MCP servers in Central Catalog", "Failed")
if output.success:
    print(output.text)

# Show deployments with runtime URIs for each MCP server
print("\n🔗 MCP Server Endpoints (from Central Catalog deployments)")
print("=" * 80)
mcp_endpoints = {}
for api in central_apis:
    if api.get('kind') == 'mcp':
        deployments = get_apic_deployments(rg1_name, rg1_apic_name, api['name'])
        for dep in deployments:
            uris = dep.get('server', {}).get('runtimeUri', [])
            source = dep.get('title', '')
            for uri in uris:
                print(f"  {api['name']:25} → {uri}  ({source})")
                mcp_endpoints[api['name']] = uri

print(f"\n📊 {len(mcp_endpoints)} MCP servers discoverable with invocation endpoints")

⚙️ Running: az apic api list -g rg-lab-apic-central-2 -n apic-central-2 --query "[?kind=='mcp'].{Name:name, Title:title, Kind:kind}" -o table 
✅ MCP servers in Central Catalog ⌚ 11:31:52.628528 :5s]
Name            Title                Kind
--------------  -------------------  ------
sentry          Sentry               mcp
intercom        Intercom             mcp
asana           Asana                mcp
plaid           Plaid                mcp
atlassian       Atlassian            mcp
linear          Linear               mcp
cloudflare      Cloudflare           mcp
paypal          Paypal               mcp
square          Square               mcp
order-mcp       Order Service MCP    mcp
calculator-mcp  Calculator MCP       mcp
catalog-mcp     Product Catalog MCP  mcp
weather-mcp     Weather MCP          mcp


🔗 MCP Server Endpoints (from Central Catalog deployments)
⚙️ Running: az apic api deployment list -g rg-lab-apic-central-2 -n apic-central-2 --api-id sentry -o json 
  sentry      

### 🔌 Dynamic connection — discover and invoke MCP servers from Central Catalog

An agent or client can:
1. Query the central API Center for `kind == 'mcp'`
2. Get the `runtimeUri` from deployments
3. Connect and invoke tools — **without knowing which APIM or RG hosts them**

In [76]:
# Dynamically discover and invoke MCP servers from the central catalog
print("🔌 Dynamic MCP Discovery + Invocation from Central Catalog")
print("=" * 65)

# Step 1: Discover MCP servers
output = utils.run(f'az apic api list -g {rg1_name} -n {rg1_apic_name} --query "[?kind==\'mcp\']" -o json', "", "")
discovered_servers = output.json_data if output.success and output.json_data else []
print(f"\n  Step 1: Discovered {len(discovered_servers)} MCP servers in central catalog")

# Step 2: Get runtime URIs
print(f"  Step 2: Resolving runtime endpoints...")
invocable = {}
for api in discovered_servers:
    deps = get_apic_deployments(rg1_name, rg1_apic_name, api['name'])
    for dep in deps:
        for uri in dep.get('server', {}).get('runtimeUri', []):
            # Runtime URI from APIC deployment IS the endpoint — don't append /mcp
            invocable[api['name']] = uri
            break  # first deployment

# Step 3: Invoke tools/list on our APIM-hosted servers (skip external 3rd-party servers)
our_gateway = rg3_gateway_url.rstrip('/')
our_servers = {k: v for k, v in invocable.items() if v.startswith(our_gateway)}
external    = {k: v for k, v in invocable.items() if not v.startswith(our_gateway)}

print(f"  Step 3: Invoking tools on {len(our_servers)} APIM-hosted servers "
      f"(skipping {len(external)} external 3rd-party servers)\n")

apim_headers = {"Ocp-Apim-Subscription-Key": rg3_api_key}
for name, endpoint in sorted(our_servers.items()):
    tools = list_mcp_tools(endpoint, label=name, extra_headers=apim_headers)
    if not tools:
        print(f"  ⚠️  {name}: no tools returned")

if external:
    print(f"\n  📋 External MCP servers (not invoked — require vendor auth):")
    for name, uri in sorted(external.items()):
        print(f"     {name:25} → {uri}")

print(f"\n📊 Discovered {len(invocable)} MCP servers, invoked {len(our_servers)} via APIM gateway")

🔌 Dynamic MCP Discovery + Invocation from Central Catalog
⚙️ Running: az apic api list -g rg-lab-apic-central-2 -n apic-central-2 --query "[?kind=='mcp']" -o json 

  Step 1: Discovered 13 MCP servers in central catalog
  Step 2: Resolving runtime endpoints...
⚙️ Running: az apic api deployment list -g rg-lab-apic-central-2 -n apic-central-2 --api-id sentry -o json 
⚙️ Running: az apic api deployment list -g rg-lab-apic-central-2 -n apic-central-2 --api-id intercom -o json 
⚙️ Running: az apic api deployment list -g rg-lab-apic-central-2 -n apic-central-2 --api-id asana -o json 
⚙️ Running: az apic api deployment list -g rg-lab-apic-central-2 -n apic-central-2 --api-id plaid -o json 
⚙️ Running: az apic api deployment list -g rg-lab-apic-central-2 -n apic-central-2 --api-id atlassian -o json 
⚙️ Running: az apic api deployment list -g rg-lab-apic-central-2 -n apic-central-2 --api-id linear -o json 
⚙️ Running: az apic api deployment list -g rg-lab-apic-central-2 -n apic-central-2 --api

---
## Part 7 — Agent Workflow: Cross-Environment Tool Chaining
---

> An agent discovers MCP servers from the **central catalog** and chains tools across **multiple APIM instances** — Weather from RG 2, Calculator from RG 3, etc.

### Simulated agent scenario:
1. **Discover** available MCP servers from central API Center
2. **Search catalog** (RG 2's APIM) → find a laptop
3. **Place order** (RG 2's APIM) → order it
4. **Calculate total** (RG 3's APIM) → compute with tax
5. **Check weather** (RG 3's APIM) → for delivery planning

In [78]:
print("🤖 Agent Workflow — Cross-Environment Tool Chaining")
print("=" * 65)

# Step 1: Agent discovers MCP servers from central catalog
print("\n📋 Step 1: Agent queries central API Center for MCP servers")
mcp_servers = {api['name']: api.get('title','') for api in discovered_servers}
for name, title in sorted(mcp_servers.items()):
    endpoint = invocable.get(name, '—')
    hosted = "APIM" if endpoint.startswith(rg3_gateway_url) else "External"
    print(f"  • {title:30} [{hosted}] → {endpoint[:70]}...")

# APIM-hosted endpoints need the subscription key
apim_headers = {"Ocp-Apim-Subscription-Key": rg3_api_key}

# Step 2: Search catalog
print("\n🔍 Step 2: Agent calls catalog MCP → search for 'laptop'")
catalog_ep = invocable.get('catalog-mcp')
if catalog_ep:
    result = call_mcp(catalog_ep, "search_products", {"query": "laptop"}, "Catalog search", apim_headers)
    if result:
        print(f"    → Found: {json.dumps(result)[:200]}")

# Step 3: Place order
print("\n📦 Step 3: Agent calls order MCP → place order")
order_ep = invocable.get('order-mcp')
if order_ep:
    result = call_mcp(order_ep, "place_order", {"product_id": "PROD-001", "quantity": 2}, "Place order", apim_headers)
    if result:
        print(f"    → Order: {json.dumps(result)[:200]}")

# Step 4: Calculate total with tax
print("\n🧮 Step 4: Agent calls calculator MCP → compute total with tax")
calc_ep = invocable.get('calculator-mcp')
if calc_ep:
    result = call_mcp(calc_ep, "calculate", {"operation": "multiply", "a": 999.99, "b": 1.08}, "Calculate total", apim_headers)
    if result:
        total = result.get('result', result) if isinstance(result, dict) else result
        print(f"    → Total with 8% tax: ${total}")

# Step 5: Check weather for delivery
print("\n🌤️ Step 5: Agent calls weather MCP → check delivery weather")
weather_ep = invocable.get('weather-mcp')
if weather_ep:
    result = call_mcp(weather_ep, "get_weather", {"city": "Seattle"}, "Weather check", apim_headers)
    if result:
        print(f"    → Weather: {json.dumps(result)[:200]}")

print("\n" + "=" * 65)
print("✅ Agent workflow complete — 4 tools chained across APIM gateway via central catalog!")

🤖 Agent Workflow — Cross-Environment Tool Chaining

📋 Step 1: Agent queries central API Center for MCP servers
  • Asana                          [External] → https://mcp.asana.com/sse...
  • Atlassian                      [External] → https://mcp.atlassian.com/v1/sse...
  • Calculator MCP                 [APIM] → https://apim-fastmcp-2.azure-api.net/calculator-mcp...
  • Product Catalog MCP            [APIM] → https://apim-fastmcp-2.azure-api.net/catalog-mcp...
  • Cloudflare                     [External] → https://docs.mcp.cloudflare.com/sse...
  • Intercom                       [External] → https://mcp.intercom.com/sse...
  • Linear                         [External] → https://mcp.linear.app/sse...
  • Order Service MCP              [APIM] → https://apim-fastmcp-2.azure-api.net/order-mcp...
  • Paypal                         [External] → https://mcp.paypal.com/sse...
  • Plaid                          [External] → https://api.dashboard.plaid.com/mcp/sse...
  • Sentry               

---
## Part 8 — Explore A2A Agents (RG 4)
---

> RG 4 (`rg-lab-a2a-demo`) contains **A2A agents** (Agent-to-Agent protocol) alongside REST APIs and MCP servers.  
> These agents expose `/.well-known/agent.json` agent cards and communicate via the [A2A protocol](https://google.github.io/A2A/).
> 
> **Architecture**: REST API → APIM proxy → A2A agent (registered in APIC with `kind: a2a`)

In [79]:
# List APIs in the A2A API Center — shows all 3 protocol types
print("📋 APIs registered in RG 4 API Center")
print("=" * 70)

output = utils.run(
    f'az apic api list -g {rg4_name} -n {rg4_apic} '
    f'--query "[].{{Name:name, Title:title, Kind:kind}}" -o table',
    "Listed APIs in A2A API Center", "Failed")
if output.success:
    print(output.text)

# Count by kind
output = utils.run(f'az apic api list -g {rg4_name} -n {rg4_apic} -o json', "", "")
if output.success and output.json_data:
    rg4_apis = output.json_data
    rg4_rest = sum(1 for api in rg4_apis if api.get('kind') == 'rest')
    rg4_mcp  = sum(1 for api in rg4_apis if api.get('kind') == 'mcp')
    rg4_a2a  = sum(1 for api in rg4_apis if api.get('kind') == 'a2a')
    print(f"\n📊 RG 4: {len(rg4_apis)} APIs — {rg4_rest} REST, {rg4_mcp} MCP, {rg4_a2a} A2A agents")

📋 APIs registered in RG 4 API Center
⚙️ Running: az apic api list -g rg-lab-a2a-demo -n apic-a2a-v2vnj5h4ckvgy --query "[].{Name:name, Title:title, Kind:kind}" -o table 
✅ Listed APIs in A2A API Center ⌚ 11:39:13.593666 :4s]
Name                   Title                        Kind
---------------------  ---------------------------  ------
paypal                 Paypal                       mcp
cloudflare             Cloudflare                   mcp
intercom               Intercom                     mcp
linear                 Linear                       mcp
plaid                  Plaid                        mcp
square                 Square                       mcp
atlassian              Atlassian                    mcp
sentry                 Sentry                       mcp
asana                  Asana                        mcp
swagger-petstore       Swagger Petstore             rest
outline-generator-api  Outline Generator API        rest
title-generator-api    Title Generator AP

### 🔍 Explore A2A agents — APIM APIs and agent cards

A2A agents are exposed through APIM just like MCP servers. Each agent serves:
- `/.well-known/agent.json` — **Agent Card** (name, description, skills, capabilities)
- `/` — A2A JSON-RPC endpoint for task submission

In [81]:
# List APIM APIs in RG 4 — shows the A2A agent APIs alongside REST
print("📋 APIM APIs in RG 4")
print("=" * 70)

# Use ARM REST to get API details including type
import subprocess
token = subprocess.run(["az", "account", "get-access-token", "--query", "accessToken", "-o", "tsv"],
                       capture_output=True, text=True, shell=True).stdout.strip()
sub_id = "31613fe0-1e9b-4a97-b771-dc48fbaa0fbb"
apim_url = f"https://management.azure.com/subscriptions/{sub_id}/resourceGroups/{rg4_name}/providers/Microsoft.ApiManagement/service/{rg4_apim}/apis?api-version=2024-06-01-preview"
resp = requests.get(apim_url, headers={"Authorization": f"Bearer {token}"})
if resp.status_code == 200:
    apis = resp.json().get('value', [])
    for api in apis:
        name = api['name']
        display = api['properties'].get('displayName', '')
        path = api['properties'].get('path', '')
        api_type = api['properties'].get('type', 'http')
        print(f"  {name:30} path=/{path:20} type={api_type:8} {display}")
    print(f"\n📊 {len(apis)} APIs in APIM ({rg4_apim})")

# Try to fetch agent cards from the A2A agents via APIM
print("\n🃏 Fetching A2A Agent Cards via APIM")
print("-" * 70)

a2a_agents = [api for api in rg4_apis if api.get('kind') == 'a2a']
for agent in a2a_agents:
    agent_name = agent['name']
    # Get deployment runtimeUri for this agent
    deps = get_apic_deployments(rg4_name, rg4_apic, agent_name)
    for dep in deps:
        for uri in dep.get('server', {}).get('runtimeUri', []):
            card_url = f"{uri}/.well-known/agent.json"
            try:
                card_resp = requests.get(card_url, timeout=10)
                if card_resp.status_code == 200:
                    card = card_resp.json()
                    print(f"\n  🤖 {card.get('name', agent_name)}")
                    print(f"     Description: {card.get('description', 'N/A')}")
                    print(f"     Version: {card.get('version', 'N/A')}")
                    skills = card.get('skills', [])
                    for skill in skills:
                        print(f"     Skill: {skill.get('name', '')} — {skill.get('description', '')}")
                    caps = card.get('capabilities', {})
                    print(f"     Streaming: {caps.get('streaming', False)}")
                else:
                    print(f"\n  ⚠️ {agent_name}: agent card returned {card_resp.status_code}")
            except Exception as e:
                print(f"\n  ⚠️ {agent_name}: {e}")
            break

📋 APIM APIs in RG 4
  outline-agent                  path=/outline-agent        type=http     Outline Generator A2A Agent
  outline-generator-api          path=/outline-generator    type=http     Outline Generator API
  summary-agent                  path=/summary-agent        type=http     Summary Generator A2A Agent
  summary-generator-api          path=/summary-generator    type=http     Summary Generator API
  title-agent                    path=/title-agent          type=http     Title Generator A2A Agent
  title-generator-api            path=/title-generator      type=http     Title Generator API

📊 6 APIs in APIM (apim-a2a-wihxr7)

🃏 Fetching A2A Agent Cards via APIM
----------------------------------------------------------------------
⚙️ Running: az apic api deployment list -g rg-lab-a2a-demo -n apic-a2a-v2vnj5h4ckvgy --api-id outline-agent -o json 

  🤖 Outline Generator Agent
     Description: An intelligent outline generator agent. I can help you create structured outlines 

---
## Part 9 — Aggregate A2A Agents to Central Catalog
---

> Sync APIs from **RG 4** (A2A agents + REST + MCP) into the **Central API Center**.  
> Uses the same `register_api_in_central()` function — it now routes `kind: a2a` to the A2A environment automatically.

In [98]:
# Aggregate from RG 4 (A2A agents + REST + MCP)
rg4_apis = get_apic_apis(rg4_name, rg4_apic)
print(f"📥 Syncing {len(rg4_apis)} APIs from RG 4 ({rg4_apic}) → Central APIC")
print("=" * 70)

rg4_synced = 0
for api in rg4_apis:
    uris = register_api_in_central(api, rg4_name, rg4_apic, "rg4")
    uri_str = uris[0] if uris else "—"
    kind = api.get('kind', '?')
    icon = {'a2a': '🤖', 'mcp': '🔌', 'rest': '🌐'}.get(kind, '📦')
    print(f"  {icon} {api['name']:25} ({kind:4}) → {uri_str}")
    rg4_synced += 1

rg4_rest = sum(1 for api in rg4_apis if api.get('kind') == 'rest')
rg4_mcp  = sum(1 for api in rg4_apis if api.get('kind') == 'mcp')
rg4_a2a  = sum(1 for api in rg4_apis if api.get('kind') == 'a2a')
print(f"\n📊 Synced {rg4_synced} APIs from RG 4 ({rg4_rest} REST, {rg4_mcp} MCP, {rg4_a2a} A2A)")

⚙️ Running: az apic api list -g rg-lab-a2a-demo -n apic-a2a-v2vnj5h4ckvgy -o json 
📥 Syncing 16 APIs from RG 4 (apic-a2a-v2vnj5h4ckvgy) → Central APIC
⚙️ Running: az rest --method PUT --url "https://management.azure.com/subscriptions/31613fe0-1e9b-4a97-b771-dc48fbaa0fbb/resourceGroups/rg-lab-apic-central-2/providers/Microsoft.ApiCenter/services/apic-central-2/workspaces/default/apis/paypal?api-version=2024-06-01-preview" --body @_temp_api_paypal.json 
⚙️ Running: az apic api version create -g rg-lab-apic-central-2 -n apic-central-2 --api-id paypal --version-id 1-0-0 --title "1.0.0" --lifecycle-stage production 
⚙️ Running: az apic api definition create -g rg-lab-apic-central-2 -n apic-central-2 --api-id paypal --version-id 1-0-0 --definition-id default --title "Default" 
⚙️ Running: az apic api deployment list -g rg-lab-a2a-demo -n apic-a2a-v2vnj5h4ckvgy --api-id paypal -o json 
  🔌 paypal                    (mcp ) → https://mcp.paypal.com/sse
⚙️ Running: az rest --method PUT --url "ht

---
## Part 10 — Unified Governance: REST + MCP + A2A in One Catalog
---

> 💡 The **Central API Center** now contains APIs from **all 3 protocol types** across 3 distributed resource groups.  
> This is the single pane of glass for discovering and governing all AI-related APIs.

### What's in the Central Catalog
| Protocol | Discovery | Invocation |
|----------|-----------|------------|
| **REST** | `kind: rest` → OpenAPI specs | Standard HTTP `GET`/`POST` |
| **MCP** | `kind: mcp` → `runtimeUri/mcp` | Streamable MCP `tools/list`, `tools/call` |
| **A2A** | `kind: a2a` → `runtimeUri/.well-known/agent.json` | A2A JSON-RPC `tasks/send` |

In [99]:
# ── 10a. Full catalog view — all protocols, all regions ──────────────────────

# Refresh the central catalog
output = utils.run(f'az apic api list -g {rg1_name} -n {rg1_apic_name} -o json', "", "")
central_apis = output.json_data if output.success and output.json_data else []

# Categorize by kind
central_rest = [a for a in central_apis if a.get("kind") == "rest"]
central_mcp  = [a for a in central_apis if a.get("kind") == "mcp"]
central_a2a  = [a for a in central_apis if a.get("kind") == "a2a"]

print(f"╔══════════════════════════════════════════════════════╗")
print(f"║   Central API Catalog — Unified Discovery Hub       ║")
print(f"╠══════════════════════════════════════════════════════╣")
print(f"║  Total APIs registered:  {len(central_apis):<26} ║")
print(f"║  ─────────────────────────────────────────────────── ║")
print(f"║  🌐 REST APIs:           {len(central_rest):<26} ║")
print(f"║  🔌 MCP Servers:         {len(central_mcp):<26} ║")
print(f"║  🤖 A2A Agents:          {len(central_a2a):<26} ║")
print(f"╚══════════════════════════════════════════════════════╝")

# Print each API with its protocol badge
print(f"\n{'Kind':<6} {'API Name':<40} {'Title'}")
print("─" * 80)
icons = {"rest": "🌐", "mcp": "🔌", "a2a": "🤖"}
for api in sorted(central_apis, key=lambda a: (a.get("kind", ""), a.get("name", ""))):
    kind = api.get("kind", "?")
    icon = icons.get(kind, "❓")
    print(f"{icon} {kind:<4} {api['name']:<40} {api.get('title', '')}")

⚙️ Running: az apic api list -g rg-lab-apic-central-2 -n apic-central-2 -o json 
╔══════════════════════════════════════════════════════╗
║   Central API Catalog — Unified Discovery Hub       ║
╠══════════════════════════════════════════════════════╣
║  Total APIs registered:  20                         ║
║  ─────────────────────────────────────────────────── ║
║  🌐 REST APIs:           4                          ║
║  🔌 MCP Servers:         13                         ║
║  🤖 A2A Agents:          3                          ║
╚══════════════════════════════════════════════════════╝

Kind   API Name                                 Title
────────────────────────────────────────────────────────────────────────────────
🤖 a2a  outline-agent                            Outline Generator A2A Agent
🤖 a2a  summary-agent                            Summary Generator A2A Agent
🤖 a2a  title-agent                              Title Generator A2A Agent
🔌 mcp  asana                                    Asan

### 10b. Dynamic A2A Agent Discovery & Invocation

> An **AI agent** (or client) can discover A2A agents from the central catalog at runtime:  
> 1. Query API Center for APIs with `kind == "a2a"`  
> 2. Get the deployment `runtimeUri` for each agent  
> 3. Fetch the **Agent Card** at `{runtimeUri}/.well-known/agent.json`  
> 4. Invoke the agent via A2A protocol (`tasks/send`)  
>
> All traffic flows through **APIM** — giving you rate limiting, auth, monitoring, and governance for free.

In [100]:
# ── 10b. Dynamic A2A agent discovery from central catalog ────────────────────

# Step 1: Query central APIC for A2A agents
output = utils.run(
    f'az apic api list -g {rg1_name} -n {rg1_apic_name} --query "[?kind==\'a2a\']" -o json', "", "")
a2a_agents = output.json_data if output.success and output.json_data else []
print(f"Found {len(a2a_agents)} A2A agents in the central catalog\n")

# Step 2: For each agent, get its deployment runtimeUri
discovered_agents = []

for agent in a2a_agents:
    api_name = agent["name"]
    deployments = get_apic_deployments(rg1_name, rg1_apic_name, api_name)

    for dep in deployments:
        runtime_uri = dep.get("server", {}).get("runtimeUri", [None])[0]
        env_id = dep.get("environmentId", "")
        env_name = env_id.split("/")[-1] if env_id else ""

        if runtime_uri and env_name == "a2a":
            discovered_agents.append({
                "name": api_name,
                "title": agent.get("title", api_name),
                "runtime_uri": runtime_uri
            })
            print(f"🤖 {api_name}")
            print(f"   Runtime: {runtime_uri}")

# Step 3: Fetch Agent Cards (via APIM gateway)
print(f"\n{'═' * 70}")
print(f"Agent Cards (fetched from .well-known/agent.json via APIM)")
print(f"{'═' * 70}")

for agent in discovered_agents:
    card_url = f"{agent['runtime_uri']}/.well-known/agent.json"
    try:
        resp = requests.get(card_url, timeout=10)
        if resp.status_code == 200:
            card = resp.json()
            print(f"\n🤖 {card.get('name', agent['name'])}")
            print(f"   Description: {card.get('description', 'N/A')}")
            print(f"   Version:     {card.get('version', 'N/A')}")
            skills = card.get("skills", [])
            if skills:
                for skill in skills:
                    print(f"   Skill:       {skill.get('name', '?')} — {skill.get('description', '')}")
            caps = card.get("capabilities", {})
            if caps:
                print(f"   Capabilities: streaming={caps.get('streaming', False)}, pushNotifications={caps.get('pushNotifications', False)}")
        else:
            print(f"\n⚠️  {agent['name']}: HTTP {resp.status_code}")
    except Exception as e:
        print(f"\n⚠️  {agent['name']}: {e}")

⚙️ Running: az apic api list -g rg-lab-apic-central-2 -n apic-central-2 --query "[?kind=='a2a']" -o json 
Found 3 A2A agents in the central catalog

⚙️ Running: az apic api deployment list -g rg-lab-apic-central-2 -n apic-central-2 --api-id outline-agent -o json 
🤖 outline-agent
   Runtime: https://apim-a2a-wihxr7.azure-api.net/outline-agent
⚙️ Running: az apic api deployment list -g rg-lab-apic-central-2 -n apic-central-2 --api-id title-agent -o json 
🤖 title-agent
   Runtime: https://apim-a2a-wihxr7.azure-api.net/title-agent
⚙️ Running: az apic api deployment list -g rg-lab-apic-central-2 -n apic-central-2 --api-id summary-agent -o json 
🤖 summary-agent
   Runtime: https://apim-a2a-wihxr7.azure-api.net/summary-agent

══════════════════════════════════════════════════════════════════════
Agent Cards (fetched from .well-known/agent.json via APIM)
══════════════════════════════════════════════════════════════════════

🤖 Outline Generator Agent
   Description: An intelligent outline gene

### 10c. Governance Through APIM — All Protocols, One Gateway

> Every API call — **REST**, **MCP**, or **A2A** — flows through **Azure API Management**.  
> APIM provides **consistent governance** regardless of the protocol:  
>
> | Governance Feature | REST | MCP | A2A |
> |---|---|---|---|
> | **Authentication** | ✅ subscription-key / OAuth2 | ✅ subscription-key / OAuth2 | ✅ subscription-key / OAuth2 |
> | **Rate Limiting** | ✅ per-subscription quotas | ✅ per-subscription quotas | ✅ per-subscription quotas |
> | **Request Logging** | ✅ App Insights | ✅ App Insights | ✅ App Insights |
> | **Token Metrics** | ✅ via emit-token-metric | ✅ via emit-token-metric | ✅ via emit-token-metric |
> | **Circuit Breaking** | ✅ backend pool | ✅ backend pool | ✅ backend pool |
> | **IP Filtering** | ✅ policy | ✅ policy | ✅ policy |
>
> The cell below queries **APIM analytics** to show invocation counts and latency across all protocol types.

In [101]:
# ── 10c. Governance metrics — APIM analytics across all protocols ────────────

from datetime import datetime, timedelta, timezone

# Query APIM analytics for the A2A resource group (RG 4) — last 7 days
end_time = datetime.now(timezone.utc)
start_time = end_time - timedelta(days=7)
time_filter = f"timestamp ge datetime'{start_time.strftime('%Y-%m-%dT%H:%M:%SZ')}' and timestamp le datetime'{end_time.strftime('%Y-%m-%dT%H:%M:%SZ')}'"

# Use ARM REST to get APIM reports by API
access_token = subprocess.run(
    ["az", "account", "get-access-token", "--query", "accessToken", "-o", "tsv"],
    capture_output=True, text=True, shell=True).stdout.strip()

report_url = (
    f"https://management.azure.com/subscriptions/{subscription_id}/resourceGroups/{rg4_name}"
    f"/providers/Microsoft.ApiManagement/service/{rg4_apim}"
    f"/reports/byApi?$filter={time_filter}&api-version=2023-09-01-preview"
)

headers = {"Authorization": f"Bearer {access_token}"}
resp = requests.get(report_url, headers=headers, timeout=30)

if resp.status_code == 200:
    reports = resp.json().get("value", [])

    print(f"╔══════════════════════════════════════════════════════════════════════╗")
    print(f"║   APIM Gateway Analytics — {rg4_apim:<40} ║")
    print(f"║   Period: last 7 days                                              ║")
    print(f"╠══════════════════════════════════════════════════════════════════════╣")
    print(f"║  {'API Name':<30} {'Calls':>8} {'Success':>8} {'Errors':>8} {'Avg ms':>8} ║")
    print(f"║  {'─'*30} {'─'*8} {'─'*8} {'─'*8} {'─'*8} ║")

    total_calls = 0
    total_success = 0
    total_errors = 0

    for r in sorted(reports, key=lambda x: x.get("callCountTotal", 0), reverse=True):
        name = r.get("name", "?")[:30]
        calls = r.get("callCountTotal", 0)
        success = r.get("callCountSuccess", 0)
        errors = r.get("callCountFailed", 0) + r.get("callCountOther", 0)
        avg_time = r.get("apiTimeAvg", 0)

        total_calls += calls
        total_success += success
        total_errors += errors

        print(f"║  {name:<30} {calls:>8} {success:>8} {errors:>8} {avg_time:>7.0f} ║")

    print(f"║  {'─'*30} {'─'*8} {'─'*8} {'─'*8} {'─'*8} ║")
    print(f"║  {'TOTAL':<30} {total_calls:>8} {total_success:>8} {total_errors:>8}          ║")
    print(f"╚══════════════════════════════════════════════════════════════════════╝")

    print(f"\n✅ APIM provides consistent governance across REST, MCP, and A2A protocols")
    print(f"   — subscription keys, rate limits, logging, and analytics — all from one gateway.")
else:
    print(f"⚠️  Could not fetch APIM reports: HTTP {resp.status_code}")
    print(f"   (This is expected if the APIM instance has no recent traffic)")

⚠️  Could not fetch APIM reports: HTTP 400
   (This is expected if the APIM instance has no recent traffic)


---
## ✅ Demo Summary
---

In [102]:
print("=" * 70)
print("  🎬  FEDERATED API GOVERNANCE DEMO — SUMMARY")
print("=" * 70)
print()
print("  Architecture:")
print(f"    RG 1 — Central APIC:   {rg1_apic_name} ({rg1_location})")
print(f"    RG 3 — FastMCP:        {rg3_apic} ({rg3_name})")
print(f"    RG 4 — A2A Agents:     {rg4_apic} ({rg4_name})")
print()
print("  API Inventory:")
_rg3_mcp = rg3_mcp if 'rg3_mcp' in dir() else '?'
print(f"    RG 3: {len(rg3_apis) if 'rg3_apis' in dir() else '?'} APIs ({_rg3_mcp} MCP)")
_rg4_rest = rg4_rest if 'rg4_rest' in dir() else '?'
_rg4_mcp  = rg4_mcp  if 'rg4_mcp'  in dir() else '?'
_rg4_a2a  = rg4_a2a  if 'rg4_a2a'  in dir() else '?'
_rg4_apis = rg4_apis if 'rg4_apis' in dir() else []
print(f"    RG 4: {len(_rg4_apis)} APIs ({_rg4_rest} REST, {_rg4_mcp} MCP, {_rg4_a2a} A2A)")
# central_rest/mcp/a2a are lists after 10a
_c_rest = len(central_rest) if isinstance(central_rest, list) else central_rest
_c_mcp  = len(central_mcp)  if isinstance(central_mcp, list)  else central_mcp
_c_a2a  = len(central_a2a)  if isinstance(central_a2a, list)  else central_a2a
print(f"    Central: {len(central_apis)} APIs ({_c_rest} REST, {_c_mcp} MCP, {_c_a2a} A2A)")
print()
print("  Key Takeaways:")
print("    • Central API Center = single pane of glass for REST + MCP + A2A discovery")
print("    • APIM = invocation layer with subscription keys, rate limits, and logging")
print("    • A2A agents are discoverable alongside MCP servers and REST APIs")
print("    • Agent cards provide self-describing metadata for A2A agents")
print("    • Agents discover from Central APIC, invoke through distributed APIMs")
print("    • New APIs auto-registered in local APIC, synced to central on demand")
print("    • Works across subscriptions, resource groups, and regions")
print("=" * 70)

  🎬  FEDERATED API GOVERNANCE DEMO — SUMMARY

  Architecture:
    RG 1 — Central APIC:   apic-central-2 (swedencentral)
    RG 3 — FastMCP:        apic-fastmcp-2-brasrj7hswmuc (rg-lab-mcp-containers-2)
    RG 4 — A2A Agents:     apic-a2a-v2vnj5h4ckvgy (rg-lab-a2a-demo)

  API Inventory:
    RG 3: 14 APIs (13 MCP)
    RG 4: 16 APIs (4 REST, 9 MCP, 3 A2A)
    Central: 20 APIs (4 REST, 13 MCP, 3 A2A)

  Key Takeaways:
    • Central API Center = single pane of glass for REST + MCP + A2A discovery
    • APIM = invocation layer with subscription keys, rate limits, and logging
    • A2A agents are discoverable alongside MCP servers and REST APIs
    • Agent cards provide self-describing metadata for A2A agents
    • Agents discover from Central APIC, invoke through distributed APIMs
    • New APIs auto-registered in local APIC, synced to central on demand
    • Works across subscriptions, resource groups, and regions


---
## 🗑️ Clean up resources

Uncomment and run to delete the resource groups created by this demo.  
RG 2 and RG 4 are from other labs and are left intact. Or use [clean-up-resources.ipynb](clean-up-resources.ipynb).

In [103]:
# Uncomment to delete demo resources (uses current IDX value):
# Note: RG 2 and RG 4 are pre-existing — NOT deleted here

# utils.run(f"az group delete --name {rg3_name} --yes --no-wait",
#           f"✅ {rg3_name} deletion initiated", "Failed")
# utils.run(f"az group delete --name {rg1_name} --yes --no-wait",
#           f"✅ {rg1_name} deletion initiated", "Failed")
# print(f"💡 To redeploy, increment IDX to {IDX + 1} in the variables cell")

# Note: RG 2 (rg-lab-mcp-demo-1-1) is NOT deleted — it was pre-existing
# Note: RG 4 (rg-lab-a2a-demo) is NOT deleted — it was pre-existing
print("⚠️  Uncomment the lines above to delete demo resources")

⚠️  Uncomment the lines above to delete demo resources
